In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls

drive/  sample_data/


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/7313Project/test_with_all.csv")
train = pd.read_csv("/content/drive/MyDrive/7313Project/train_with_all.csv")
testn = pd.read_csv("/content/drive/MyDrive/7313Project/test_with_all.csv")
target = pd.read_csv("/content/drive/MyDrive/7313Project/target.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
train.drop(train.iloc[:, [0,1]], axis=1, inplace=True)
test.drop(test.iloc[:, [0,1]], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [ ]:
import tensorflow as tf
from tensorflow import keras
train['is_cup'] = labelencoder.fit_transform(train['is_cup'])
test['is_cup'] = labelencoder.fit_transform(test['is_cup'])
target['target'].replace({"home": 0, "draw": 1, "away":2}, inplace=True)
train.drop(train.filter(regex='recorded_matches').columns, axis=1, inplace = True)
test.drop(test.filter(regex='recorded_matches').columns, axis=1, inplace = True)
train_x = train.copy()
train_y = keras.utils.to_categorical(target['target'], num_classes=3)
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
#fit scaler on training data
norm = StandardScaler().fit(train_x)
#transform training data
xtrain_norm = norm.transform(train_x)
#transform testing data
test_norm = norm.transform(test)

In [ ]:
!pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
xtrain_norm.shape

(110938, 304)

In [ ]:
train_y.shape

(110938, 3)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# Initialize a Sequential model. Sequential means that layers are added in sequence one after the other.
model = tf.keras.Sequential([
      # Define the activation function for the first fully connected (Dense) layer.
      layers.Dense(200, activation=tf.nn.relu),
      layers.Dropout(rate=0.2),
      # Define the activation function for the first fully connected (Dense) layer.
      layers.Dense(100, activation=tf.nn.relu),
      keras.layers.Dropout(rate=0.2),
      # Define the second Dense layer to output the classification probabilities
      layers.Dense(3, activation=tf.nn.softmax) 
  ])


# Compile the model. Here we specify the type of gradient descent algorithm to use,
# the loss function, and the types of metrics that we want to calculate for each epoch
model.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=[keras.metrics.Precision(), keras.metrics.Recall(), tfa.metrics.F1Score(num_classes=3)])
EPOCHS=300
history = model.fit(xtrain_norm, train_y, epochs=EPOCHS)

Epoch 1/300
3467/3467 [==============================] - 40s 9ms/step - loss: 0.6003 - precision: 0.5941 - recall: 0.2128 - f1_score: 0.3686
Epoch 2/300
3467/3467 [==============================] - 23s 7ms/step - loss: 0.5935 - precision: 0.6137 - recall: 0.2213 - f1_score: 0.3658
Epoch 3/300
3467/3467 [==============================] - 24s 7ms/step - loss: 0.5922 - precision: 0.6201 - recall: 0.2245 - f1_score: 0.3653
Epoch 4/300
3467/3467 [==============================] - 24s 7ms/step - loss: 0.5907 - precision: 0.6244 - recall: 0.2256 - f1_score: 0.3699
Epoch 5/300
3467/3467 [==============================] - 24s 7ms/step - loss: 0.5897 - precision: 0.6252 - recall: 0.2290 - f1_score: 0.3724
Epoch 6/300
3467/3467 [==============================] - 23s 7ms/step - loss: 0.5878 - precision: 0.6321 - recall: 0.2315 - f1_score: 0.3752
Epoch 7/300
3467/3467 [==============================] - 23s 7ms/step - loss: 0.5861 - precision: 0.6366 - recall: 0.2356 - f1_score: 0.3801
Epoch 8/300
3

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               61000     
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 81,403
Trainable params: 81,403
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred = model.predict(xtrain_norm)
y_pred

In [ ]:
classes = np.argmax(y_pred, axis = 1)
print(classes)

In [ ]:
train_acc = np.mean(classes == target['target'])*100
train_acc

In [ ]:
from sklearn.metrics import f1_score
f1_score(target['target'], classes, average='micro')

In [ ]:
from mlxtend.evaluate import confusion_matrix
confmat = confusion_matrix(target['target'], classes)

print(confmat)

In [ ]:
prediction_1 = model.predict(test_norm)
prediction_1

In [ ]:
submission = pd.concat([pd.DataFrame(testn['id']), pd.DataFrame(prediction_1, columns=['home', 'draw', 'away'])[['home', 'draw', 'away']]], axis=1)
submission.to_csv('/content/drive/MyDrive/7313Project/submission.csv', index=False)